In [1]:
import os
from itertools import count
import mne
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
%matplotlib osx

In [3]:
# %matplotlib widget

In [4]:
root = '/Volumes/My Passport/AI_Research/data/physionet.org/files/chbmit/1.0.0/chb01/'

In [5]:
# Summary
# File Name: chb01_03.edf
# File Start Time: 13:43:04
# File End Time: 14:43:04
# Number of Seizures in File: 1
# Seizure Start Time: 2996 seconds
# Seizure End Time: 3036 seconds
raw = mne.io.read_raw_edf(input_fname=os.path.join(root, os.listdir(root)[3]), preload=False, verbose='Error')

In [7]:
seizure_start_time = 2996 # in seconds
seizure_end_time = 3036
seizure_duration = seizure_end_time - seizure_start_time
print(f'Seizure duration: {seizure_duration}s')

Seizure duration: 40s


In [ ]:
interictal_start = 720 # interictal sample start (in seconds)
interictal_end

In [6]:

ch_names = raw.info['ch_names'] # array of channel names
EEG, times = raw.copy().crop(tmin=2996-10, tmax=2996+20).get_data(picks=['eeg'], return_times=True, units='uV')

In [7]:
# plot signal streaming as if 'real time'
ch_idx = 0 # select channel by index
fig = plt.figure()
ax = plt.axes(xlim=(np.min(times), np.max(times)), 
              ylim=(np.min(EEG[ch_idx,:])*1.5, np.max(EEG[ch_idx,:])*1.5))

xs = []
ys = []
line, = ax.plot([], [], 'k', lw=2)
ax.axvline(x=10, color='gray', ls='--') # mark seizure onset

def animate(i, xs, ys):
    Ik = (i-1)*N+1
    timebuf = times[i:i+5]
    sigbuf = EEG[0,i:i+5]
    
    
    xs.append(timebuf)
    ys.append(sigbuf)
    line.set_data(xs, ys)
    return line,

anim = animation.FuncAnimation(fig, animate, fargs=(xs, ys),
                               interval=5, blit=True)
plt.xlabel('Time, s')
plt.ylabel(r'Potential Difference, $\mu V$')
plt.title(f'Electrode: {ch_names[ch_idx]}')
plt.show()

In [8]:
fs = raw.info['sfreq']  # sampling frequency
fp = 0.5                # prediction frequency
N = 512                 # fs = N*fp (N must be a natural number)
fs

256.0

In [9]:
Ik_init = (1-1)*N+1
print(Ik_init)
for i in range(2, 6):
    Ik = (i-1)*N+1
    print(f'i:{i}, Ik:{Ik}, dif:{Ik - Ik_init}')
    Ik_init = Ik

1
i:2, Ik:513, dif:512
i:3, Ik:1025, dif:512
i:4, Ik:1537, dif:512
i:5, Ik:2049, dif:512


In [10]:
X = EEG[0,:]
np.rint(len(X)/N).astype(int)

15

In [52]:
print(f'Window length: {N/fs}s')
X = EEG[0,:]
Ik_buf = (1-1)*N+1
ARs = []
for k in range(2, np.rint(len(X)/N).astype(int)):
    Ik = (k-1)*N+1
    print('Ik:', Ik)
    sigbuf = X[Ik_buf:Ik]
    print('sigbuf.shape:', sigbuf.shape)
    print(f'Window {k-1} size:', sigbuf.shape)
    AR_order = 6 # AR(6) model
    A = np.random.randn(AR_order) # a_j: init AR coeffs from Norm. Dist.
    AR_sigbuf = sigbuf[len(sigbuf)-6:len(sigbuf)] # x_t-j
    print('AR_sigbuf.shape:', AR_sigbuf.shape)
    E = np.random.randn(AR_order) # e_t
    print('E.shape:', E.shape)
    AR = A @ AR_sigbuf + E
    print('Raw Buffer:', AR_sigbuf)
    print('AR:', AR)
    ARs.append(AR)
    Ik_buf = Ik
    print('---')

Window length: 2.0s
Ik: 513
sigbuf.shape: (512,)
Window 1 size: (512,)
AR_sigbuf.shape: (6,)
E.shape: (6,)
Raw Buffer: [41.61172161 42.002442   47.08180708 51.77045177 50.5982906  47.86324786]
AR: [160.652173   158.70448398 160.58130419 159.70126726 160.06063391
 160.79234221]
---
Ik: 1025
sigbuf.shape: (512,)
Window 2 size: (512,)
AR_sigbuf.shape: (6,)
E.shape: (6,)
Raw Buffer: [-16.60561661  -8.4004884    1.36752137  11.91697192  20.51282051
  16.21489621]
AR: [-0.4151501  -1.78112148 -2.33530856 -1.73877804 -3.12222408  0.04640154]
---
Ik: 1537
sigbuf.shape: (512,)
Window 3 size: (512,)
AR_sigbuf.shape: (6,)
E.shape: (6,)
Raw Buffer: [  3.71184371   6.05616606   6.44688645   0.97680098  -9.18192918
 -21.29426129]
AR: [24.50477309 25.37712219 24.84707474 24.87427364 25.45490929 23.61649177]
---
Ik: 2049
sigbuf.shape: (512,)
Window 4 size: (512,)
AR_sigbuf.shape: (6,)
E.shape: (6,)
Raw Buffer: [2.93040293 0.1953602  4.88400488 8.00976801 8.79120879 9.96336996]
AR: [22.51131124 23.7239

In [12]:
# TODO: use the data in each window to compute minimising parameters of least-squares regression.